In [2]:
############### FIRST ASSIGMENT ################################
# Import all necessary libraries
import requests
import lxml.html as lh
import pandas as pd
import folium 
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

# Used at third assigment
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [3]:
# Get the information from wikipedia post url and keep only de table data (<tr>)
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url)
doc = lh.fromstring(r.content)
tr_elements = doc.xpath('//tr')

In [4]:
# Get the headers

col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


In [5]:
# Get every value from the table

for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    i=0
    for t in T.iterchildren():
        data=t.text_content() 
        if len(T)!=3:
            break
        col[i][1].append(data)
        i+=1

In [6]:
# Create our Dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

# Delete incorrect endlines copied at scraping process
df = df.replace('\n','', regex=True)
df.columns = ('Postcode', 'Borough', 'Neighborhood')

# Ignore rows where BOROUGH is Not assigned
df = df[df['Borough'] != 'Not assigned']

# If Neighbourhood is 'Not assigned', Neighborhood becomes Borough
df['Neighborhood'] = df.apply(lambda x: x['Borough'] if x['Neighborhood'] == 'Not assigned' else x['Neighborhood'], axis=1)

# Reset index after droping rows
df.reset_index(drop=True, inplace = True)

# Delete last wrong row
df.drop(index = 211, inplace = True)
df.head(12)



,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [7]:
#Group by Postcode and Borough and concatenate the Neighborhood values separated by commas.
df2= df.groupby(["Postcode", "Borough"], as_index=False)['Neighborhood'].apply(lambda tags: ', '.join(tags))

# Add index again
df2 = df2.reset_index()

# Set Neighborhood as a header
df2.columns = ('Postcode', 'Borough', 'Neighborhood')
df2.head(12)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [8]:
#Get the dimensions
df2.shape

############### END OF FIRST ASSIGMENT ################################

(103, 3)

In [9]:
############### SECOND ASSIGMENT ################################

# Create a dataframe with latitude and longitude of every Poscode
df_geo_coor = pd.read_csv("http://cocl.us/Geospatial_data")

# Rename Postal Code column in order to use merge
df_geo_coor.rename(columns={"Postal Code": "Postcode"}, inplace = True)

# Show some values to check if everything its ok
df_geo_coor.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# Merge our Neighborhood dataframe with the lat/long dataframe by Postcode field
df_final = pd.merge(df2, df_geo_coor, on="Postcode")

# Show final results
df_final.head(12)


############### END OF SECOND ASSIGMENT ################################

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [11]:
############### THIRD ASSIGMENT ################################

# install Folium
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab



Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


folium-0.5.0         | 45 KB     | #################################

In [12]:
# Get Toronto location
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)

# Paint Toronto Map
map_toronto = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

# Lets create a new dataframe to avoid missunderstandings
df_toronto = df_final

# Add Neighborhood markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

After showing all Neighborhoods of Toronto, we will focus now on getting the main venues 300 meters around every neighborhood in order to classify them in clusters.

Using Foursquare we will get the venues as we did in the LAB, and after this we will find the top8 most commoun venues by neighborhood.

In [13]:
# @hidden cell
CLIENT_ID = 'OBBPSPRMQA40DOLVKLGVKOPZ304P1FPAZM1V2KB25AQRXUHE' # your Foursquare ID
CLIENT_SECRET = 'GKR2VVLK0C2VJ4CUCMLJ042CJPHB2O3WQFE3ZIZPI13WIFAF' # your Foursquare Secret
VERSION = '20190909' # Foursquare API version

In [63]:
# This function returns
def getNearbyVenues(names, latitudes, longitudes, radius=300):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            50)
            
        # GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

# We change Neighborhood field name becasuse there is a venue called Neighborhood too, which conflicts with this field.
toronto_venues.rename(columns={"Neighborhood": "Neighborhood_main_field"}, inplace = True)

# Show results
toronto_venues.head()

,Neighborhood_main_field,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category
0,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,Electronics Store
1,Woburn,43.770992,-79.216917,Korean Grill House,Korean Restaurant
2,Cedarbrae,43.773136,-79.239476,Federick Restaurant,Hakka Restaurant
3,Cedarbrae,43.773136,-79.239476,Drupati's Roti & Doubles,Caribbean Restaurant
4,Cedarbrae,43.773136,-79.239476,Thai One On,Thai Restaurant


In [64]:
#Create a dataframe showing how many venue catagories has every Neighborhood
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add again Neighborhood field as first column
toronto_onehot['Neighborhood_main_field'] = toronto_venues['Neighborhood_main_field'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood_main_field').mean().reset_index()
toronto_grouped

,Neighborhood_main_field,Accessories Store,Adult Boutique,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Art Gallery,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.00,0.0,0.0,0.0,0.0,0.060000,0.000000,0.00,...,0.040000,0.00,0.000000,0.000000,0.0000,0.000000,0.00,0.00,0.00,0.000000
1,Agincourt,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.00,0.00,0.00,0.000000
2,"Alderwood, Long Branch",0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.00,0.00,0.00,0.000000
3,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,...,0.000000,0.00,0.066667,0.000000,0.0000,0.000000,0.00,0.00,0.00,0.000000
4,Bayview Village,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.00,0.00,0.00,0.000000
5,"Bedford Park, Lawrence Manor East",0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.00,0.00,0.00,0.000000
6,Berczy Park,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.00,0.00,0.00,0.000000
7,"Birch Cliff, Cliffside West",0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.00,0.00,0.00,0.000000
8,"Bloordale Gardens, Eringate, Markland Wood, Ol...",0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.00,0.00,0.00,0.000000
9,"Brockton, Exhibition Place, Parkdale Village",0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00,...,0.000000,0.00,0.000000,0.076923,0.0000,0.000000,0.00,0.00,0.00,0.000000


In [53]:
# This function returns venoes sorted
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


# Create a column for TOP8
num_top_venues = 8
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a dataframe with the most common veoues for every Neighborhood
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood_main_field']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,"Adelaide, King, Richmond",Steakhouse,American Restaurant,Coffee Shop,Pizza Place,Japanese Restaurant,Seafood Restaurant,Bar,Sushi Restaurant
1,Agincourt,Breakfast Spot,Sandwich Place,Food,Flower Shop,Fish Market,Fish & Chips Shop,Field,Festival
2,"Alderwood, Long Branch",Pizza Place,Gym,Pub,Coffee Shop,Pharmacy,Hakka Restaurant,Field,Fast Food Restaurant
3,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Sushi Restaurant,Restaurant,Supermarket,Fast Food Restaurant,Middle Eastern Restaurant,Sandwich Place,Deli / Bodega
4,Bayview Village,Shopping Plaza,Yoga Studio,Food,Fish Market,Fish & Chips Shop,Field,Festival,Fast Food Restaurant


In [54]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

# Delete Neighborhood field bevause we need only numeric fields
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood_main_field', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:8] 

array([3, 3, 3, 3, 3, 3, 3, 4], dtype=int32)

In [55]:
# Add Clusters Labels as a column
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# Merge the dataframe with latitude and longitude of every neighborhood with the one with me most common venues
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# Drop NaN values
toronto_merged.dropna(subset=['Cluster Labels'], axis= 0, inplace= True) 

# Reset index
toronto_merged.reset_index(drop = True, inplace = True)

# Show results
toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,3.0,Electronics Store,Yoga Studio,Home Service,Flower Shop,Fish Market,Fish & Chips Shop,Field,Festival
1,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Korean Restaurant,Yoga Studio,Food Court,Flower Shop,Fish Market,Fish & Chips Shop,Field,Festival
2,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3.0,Thai Restaurant,Hakka Restaurant,Athletics & Sports,Caribbean Restaurant,Bank,Ethiopian Restaurant,Flower Shop,Fish Market
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,3.0,Playground,Convenience Store,Flower Shop,Fish Market,Fish & Chips Shop,Field,Festival,Fast Food Restaurant
4,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,3.0,Convenience Store,Home Service,Flower Shop,Fish Market,Fish & Chips Shop,Field,Festival,Fast Food Restaurant


In [56]:
# Get Toronto location again
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)

# Paint Toronto Map
map_clusters = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add Neighborhood markers to map
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

As result of our clustering we got 5 different kinds of Neighborhood.
Cluster 0 = Red
Cluster 1 = Purple
Cluster 2 = Blue
Cluster 3 = Green
Cluster 4 = Orange

Now I will try to detect whats the most representative of every cluster

In [57]:
# CLUSTER 0
# First Cluster hightlights are PARKS and YOGA STUDIOS. It also remarkable te lack of RESTAURANTS around.

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
17,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Yoga Studio,Electronics Store,Flower Shop,Fish Market,Fish & Chips Shop,Field,Festival
30,M4J,East York,East Toronto,43.685347,-79.338106,0.0,Park,Intersection,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Field,Festival
38,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0.0,Park,Yoga Studio,Electronics Store,Flower Shop,Fish Market,Fish & Chips Shop,Field,Festival
40,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0.0,Building,Park,Yoga Studio,Electronics Store,Fish Market,Fish & Chips Shop,Field,Festival
82,M9N,York,Weston,43.706876,-79.518188,0.0,Park,Yoga Studio,Electronics Store,Flower Shop,Fish Market,Fish & Chips Shop,Field,Festival
84,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724,0.0,Park,Bus Line,Yoga Studio,Electronics Store,Fish Market,Fish & Chips Shop,Field,Festival


In [58]:
# CLUSTER 1
# Second Cluster its made up only by two Neighbohoods, and it is focused around SPORTS, with BASEBALL, TENNIS and other kind of FIELDS.

toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
18,M3B,North York,Don Mills North,43.745906,-79.352188,1.0,Baseball Field,Tennis Court,Yoga Studio,Ethiopian Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Field
22,M3M,North York,Downsview Central,43.728496,-79.495697,1.0,Baseball Field,Yoga Studio,Food Court,Flower Shop,Fish Market,Fish & Chips Shop,Field,Festival


In [59]:
# CLUSTER 2
# Third Cluster its also made up by only two Neighbohoods, and the most remarkable are their LOCKSMITH.

toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
54,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,2.0,Locksmith,Yoga Studio,Food Court,Flower Shop,Fish Market,Fish & Chips Shop,Field,Festival
78,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509,2.0,Locksmith,Yoga Studio,Food Court,Flower Shop,Fish Market,Fish & Chips Shop,Field,Festival


In [60]:
# CLUSTER 3
# Fourth Cluster is the biggest cluster, and it has any kind of RESTAURANTS, BARS and COFFEE SHOPS.

toronto_merged.loc[toronto_merged['Cluster Labels'] == 3]

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,3.0,Electronics Store,Yoga Studio,Home Service,Flower Shop,Fish Market,Fish & Chips Shop,Field,Festival
1,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Korean Restaurant,Yoga Studio,Food Court,Flower Shop,Fish Market,Fish & Chips Shop,Field,Festival
2,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3.0,Thai Restaurant,Hakka Restaurant,Athletics & Sports,Caribbean Restaurant,Bank,Ethiopian Restaurant,Flower Shop,Fish Market
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,3.0,Playground,Convenience Store,Flower Shop,Fish Market,Fish & Chips Shop,Field,Festival,Fast Food Restaurant
4,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,3.0,Convenience Store,Home Service,Flower Shop,Fish Market,Fish & Chips Shop,Field,Festival,Fast Food Restaurant
5,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,3.0,American Restaurant,Motel,Yoga Studio,Electronics Store,Flower Shop,Fish Market,Fish & Chips Shop,Field
7,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,3.0,Brewery,Indian Restaurant,Yoga Studio,Electronics Store,Flower Shop,Fish Market,Fish & Chips Shop,Field
8,M1S,Scarborough,Agincourt,43.794200,-79.262029,3.0,Breakfast Spot,Sandwich Place,Food,Flower Shop,Fish Market,Fish & Chips Shop,Field,Festival
9,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302,3.0,Fast Food Restaurant,Shopping Mall,Bank,Yoga Studio,Ethiopian Restaurant,Flower Shop,Fish Market,Fish & Chips Shop
10,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389,3.0,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Gym,Sandwich Place,Discount Store,Grocery Store,Pizza Place


In [61]:
# CLUSTER 4
# Fifht Cluster is the last one, and it is mainly focused on ITALIAN RESTAURANTS.

toronto_merged.loc[toronto_merged['Cluster Labels'] == 4]

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
6,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,4.0,Pizza Place,Farm,Café,Electronics Store,Flower Shop,Fish Market,Fish & Chips Shop,Field
62,M6B,North York,Glencairn,43.709577,-79.445073,4.0,Pizza Place,Asian Restaurant,Sushi Restaurant,Japanese Restaurant,Dance Studio,Deli / Bodega,Fish Market,Fish & Chips Shop
81,M9L,North York,Humber Summit,43.756303,-79.565963,4.0,Pizza Place,Yoga Studio,Flower Shop,Fish Market,Fish & Chips Shop,Field,Festival,Fast Food Restaurant
83,M9P,Etobicoke,Westmount,43.696319,-79.532242,4.0,Pizza Place,Coffee Shop,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Field,Festival
